In [1]:
"""Log a pinhole and a random image."""

import numpy as np
import rerun as rr

rr.init("rerun_example_pinhole", spawn=True)
rng = np.random.default_rng(12345)

image = rng.uniform(0, 255, size=[3, 3, 3])
rr.log("world/image", rr.Pinhole(focal_length=3, width=3, height=3))
rr.log("world/image", rr.Image(image))

[2025-03-09T00:10:50Z INFO  re_sdk::spawn] A process is already listening at this address. Assuming it's a Rerun Viewer. addr=0.0.0.0:9876


In [1]:
import numpy as np
import rerun as rr
from loaders.pointodyssey import PointOdyssey
import loaders.utils.geometry as geo

dataset = PointOdyssey(dset="sample")

# data/pointodyssey/sample/scene_d78_0318_3rd
seq_path = "data/pointodyssey/sample/scene_d78_0318_3rd"
frame_info = dataset.get_frame_info(seq_path, 0)
frame_info.keys()

loading pointodyssey dataset...


dict_keys(['image', 'world_pc_valid', 'cam', 'depth_map'])

In [2]:
from scipy.spatial.transform import Rotation

def get_camera_center(extrinsics):
    R = extrinsics[:3, :3]
    t = extrinsics[:3, 3]
    return -R.T @ t  # Convert world-to-camera translation to camera center in world coordinates

def decompose_extrinsics(extrinsics):
    translation = get_camera_center(extrinsics)
    rotation_matrix = extrinsics[:3, :3]
    quaternion = Rotation.from_matrix(rotation_matrix).as_quat()
    return translation, quaternion  # Pass as NumPy array (not a Rerun object)

In [27]:
intrinsics, extrinsics = frame_info["cam"]
image = frame_info["image"]

fx, fy = intrinsics[0, 0], intrinsics[1, 1]
cx, cy = intrinsics[0, 2], intrinsics[1, 2]
img_height, img_width = image.shape[:2]

translation, quaternion = decompose_extrinsics(extrinsics)
translation, quaternion = translation, quaternion
translation, quaternion = tuple(translation), tuple(quaternion)

print(fx, fy)
print(cx, cy)
print(img_height, img_width)
print("translation:", translation)
print("quaternion:", quaternion)
print("translation type:", type(translation))
print("quaternion type:", type(quaternion))

576.0 576.0
480.0 270.0
540 960
translation: (np.float32(2.5996242), np.float32(1.8381889), np.float32(-2.8234172))
quaternion: (np.float64(-0.3577987302545975), np.float64(-0.16442011555390143), np.float64(0.916967893776276), np.float64(0.06415587278604913))
translation type: <class 'tuple'>
quaternion type: <class 'tuple'>


In [29]:
rr.spawn()

rr.log(f"pinhole1", rr.Pinhole(
    focal_length=(fx, fy),
    principal_point=(cx, cy),
    resolution=(img_width, img_height)
))
rr.log(f"pinhole1", rr.Image(image))

rr.log("pinhole2", rr.Transform3D(translation=translation, quaternion=quaternion))
rr.log(f"pinhole2", rr.Pinhole(
    focal_length=(fx, fy),
    principal_point=(cx, cy),
    resolution=(img_width, img_height)
))
rr.log(f"pinhole2", rr.Image(image))

[2025-03-09T00:26:26Z INFO  re_sdk::spawn] A process is already listening at this address. Assuming it's a Rerun Viewer. addr=0.0.0.0:9876


In [33]:
rr.spawn()

for i in range(20):
    frame_info = dataset.get_frame_info(seq_path, i)
    intrinsics, extrinsics = frame_info["cam"]
    image = frame_info["image"]
    
    fx, fy = intrinsics[0, 0], intrinsics[1, 1]
    cx, cy = intrinsics[0, 2], intrinsics[1, 2]
    img_height, img_width = image.shape[:2]
    translation, quaternion = decompose_extrinsics(extrinsics)
    # translation, quaternion = tuple(translation), tuple(quaternion)
    rr.log(f"pinhole{i}", rr.Transform3D(translation=translation, quaternion=quaternion))
    rr.log(f"pinhole{i}", rr.Pinhole(
        focal_length=(fx, fy),
        principal_point=(cx, cy),
        resolution=(img_width, img_height)
    ))
    rr.log(f"pinhole{i}", rr.Image(image))

[2025-03-09T00:30:44Z INFO  re_sdk::spawn] A process is already listening at this address. Assuming it's a Rerun Viewer. addr=0.0.0.0:9876


In [5]:
frame_info = dataset.get_frame_info(seq_path, 0)
world_pc_valid = frame_info["world_pc_valid"]
cam = frame_info["cam"]
world_pc = np.asarray(world_pc_valid[:, :3])
image = frame_info["image"]
intrinsics, extrinsics = cam
fx, fy = intrinsics[0, 0], intrinsics[1, 1]
cx, cy = intrinsics[0, 2], intrinsics[1, 2]
img_height, img_width = image.shape[:2]

translation, quaternion = decompose_extrinsics(extrinsics)

In [9]:

rr.init("my_app", spawn=True)
rr.spawn()
rr.log("pc", rr.Points3D(positions=world_pc))
rr.log("pc", rr.Transform3D(translation=translation, quaternion=quaternion))
rr.log("camera", rr.Pinhole(  # Use the same entity for pinhole and image
    focal_length=(fx, fy),
    principal_point=(cx, cy),
    resolution=(img_width, img_height)
))
rr.log("camera", rr.Image(image))  # Now properly linked to the Pinhole

[2025-03-09T00:51:09Z INFO  re_sdk::spawn] A process is already listening at this address. Assuming it's a Rerun Viewer. addr=0.0.0.0:9876
[2025-03-09T00:51:09Z INFO  re_sdk::spawn] A process is already listening at this address. Assuming it's a Rerun Viewer. addr=0.0.0.0:9876


In [3]:
frame_info = dataset.get_frame_info(seq_path, 0)
world_pc_valid = frame_info["world_pc_valid"]
cam = frame_info["cam"]
world_pc = np.asarray(world_pc_valid[:, :3])
camera_pc = geo.world_pc_to_cam_pc(world_pc, cam)
image = frame_info["image"]
intrinsics, extrinsics = cam
fx, fy = intrinsics[0, 0], intrinsics[1, 1]
cx, cy = intrinsics[0, 2], intrinsics[1, 2]
img_height, img_width = image.shape[:2]

translation, quaternion = decompose_extrinsics(extrinsics)

In [5]:

rr.init("my_app", spawn=True)
rr.spawn()
rr.log("pc", rr.Points3D(positions=camera_pc))
# rr.log("camera", rr.Transform3D(translation=translation, quaternion=quaternion))
rr.log("camera", rr.Pinhole(  # Use the same entity for pinhole and image
    focal_length=(fx, fy),
    principal_point=(cx, cy),
    resolution=(img_width, img_height)
))
rr.log("camera", rr.Image(image))  # Now properly linked to the Pinhole

[2025-03-09T01:29:48Z INFO  re_sdk::spawn] A process is already listening at this address. Assuming it's a Rerun Viewer. addr=0.0.0.0:9876
[2025-03-09T01:29:48Z INFO  re_sdk::spawn] A process is already listening at this address. Assuming it's a Rerun Viewer. addr=0.0.0.0:9876


In [ ]:
frame_info_l = dataset.get_frame_info(seq_path, 0)
frame_info_r = dataset.get_frame_info(seq_path, 0)

point_maps, motion_maps, frame_infos, ref_idx = dataset.get_point_map_and_motion_map([frame_info_l, frame_info_r], 0)
cam = frame_infos[ref_idx]["cam"]
image = frame_infos[ref_idx]["image"]
intrinsics, extrinsics = cam
fx, fy = intrinsics[0, 0], intrinsics[1, 1]
cx, cy = intrinsics[0, 2], intrinsics[1, 2]
img_height, img_width = image.shape[:2]

translation, quaternion = decompose_extrinsics(extrinsics)